In [ ]:
import pandas as pd
import numpy as np
import json
from collections import Counter
import copy

In [3]:
# Datasets and list of synonyms path
commit_sharing = "./20231012_230826_commit_sharings.json"
pull_request_sharing = "./20231012_233628_pr_sharings.json"
issue_sharing = "./20231012_235128_issue_sharings.json"
file_sharings = "./20231012_234250_file_sharings.json"
discussion_sharings = "./20231012_235320_discussion_sharings.json"

path_list_synonym = "./synonyms_for_normalization.txt"

In [ ]:
jsons_list = [commit_sharing,pull_request_sharing,issue_sharing,file_sharings,discussion_sharings]
jsons_file = ['commit_sharing','pull_request_sharing','issue_sharing','file_sharings','discussion_sharings']

In [ ]:
# Abra o arquivo json

# 0 = commit_sharing, 1 = pull_request_sharing, 2 = issue_sharing, 3 = file_sharings, 4 = discussion_sharings
fileNumber = 4

with open(jsons_list[fileNumber], 'r') as f:
    dados1 = json.load(f)

json_processado_output = jsons_file[fileNumber]+'.json'

# Normalisation

In [ ]:
# Pre-processing synonym_list file
with open(path_list_synonym, 'r') as arquivo:
    # dict to store the synonyms
    synonyms = {}
    for linha in arquivo:
        chave, valor = linha.strip().split('=')

        synonyms[chave.strip()] = valor.strip().split(',')


In [ ]:

for elementos in dados1["Sources"]:
  if elementos["RepoLanguage"] == None :
      elementos["RepoLanguage"] = "none"
  else:
    for linguagem, alias in synonyms.items():
      if elementos["RepoLanguage"] in alias:
        elementos["RepoLanguage"] = linguagem



for elementos in dados1["Sources"]:
  for elemento in elementos["ChatgptSharing"]:
    if "Conversations" in elemento and elemento["Conversations"]:
      for conversation in elemento["Conversations"]:
        if "ListOfCode" in conversation and conversation["ListOfCode"]:
          for snipetType in conversation["ListOfCode"]:
            if snipetType["Type"] == None :
              snipetType["Type"] = "none"
            else:
              for linguagem, alias in synonyms.items():
                if  snipetType["Type"] in alias:
                  snipetType["Type"] = linguagem


In [ ]:

# write processed json
with open(jsons_file[j]+'.json', 'w') as arquivo:
  json.dump(dados1, arquivo)

# Data Extraction

In [ ]:
repoLang = []
for elementos in dados1["Sources"]:
  repoLang.append(elementos["RepoLanguage"])


In [ ]:
contagem = Counter(repoLang)
contagem

contagemDict = dict(contagem)

In [ ]:
df0 = pd.DataFrame(list(contagemDict.items()), columns=['Language', 'Count'])

In [ ]:
repoByLanguage = {}

for key in contagem.keys():
  repoByLanguage.setdefault(key, [])
  for elementos in dados1["Sources"]:
    if elementos["RepoLanguage"] == key:
      repoByLanguage[key].append(elementos["RepoName"])


In [ ]:
df1 = pd.DataFrame(list(repoByLanguage.items()), columns=['Language', 'Repos'])

In [ ]:
repoByLanguageCount = copy.deepcopy(repoByLanguage)

for chave, lista in repoByLanguage.items():
    repoByLanguage[chave] = list(set(lista))
    repoByLanguageCount[chave] = len(list(set(lista)))

In [ ]:
df2 = pd.DataFrame(list(repoByLanguage.items()), columns=['Language', 'UniqueRepos'])
df3 = pd.DataFrame(list(repoByLanguageCount.items()), columns=['Language', 'UniqueReposCount'])

## CountChat

In [ ]:
chatByLanguage = {}
for elementos in dados1["Sources"]:
  chatByLanguage.setdefault(elementos["RepoLanguage"], [])
  for elemento in elementos["ChatgptSharing"]:
    chatByLanguage[elementos["RepoLanguage"]].append(elemento["URL"])



In [ ]:
# chatByLanguage["None"] = chatByLanguage[None]
# del chatByLanguage[None]

In [ ]:
df4 = pd.DataFrame(list(chatByLanguage.items()), columns=['Language', 'Chat'])
# df.to_csv('chatByLanguage.csv', index=False)

In [ ]:
chatByLanguageCount = copy.deepcopy(chatByLanguage)
for chave, lista in chatByLanguage.items():
    chatByLanguage[chave] = list(set(lista))
    chatByLanguageCount[chave] = len(list(set(lista)))

In [ ]:
chatByLanguage
df5 = pd.DataFrame(list(chatByLanguage.items()), columns=['Language', 'UniqueChat'])
df6 = pd.DataFrame(list(chatByLanguageCount.items()), columns=['Language', 'UniqueChatCount'])
# df.to_csv('uniqueChatByLanguage.csv', index=False)

## Code Snipets

In [ ]:
snipetsByLanguage = {}
for elementos in dados1["Sources"]:
  snipetsByLanguage.setdefault(elementos["RepoLanguage"], [])
  for elemento in elementos["ChatgptSharing"]:
    if "Conversations" in elemento and elemento["Conversations"]:
      for conversation in elemento["Conversations"]:
        if "ListOfCode" in conversation and conversation["ListOfCode"]:
          for snipetType in conversation["ListOfCode"]:
            snipetsByLanguage[elementos["RepoLanguage"]].append(snipetType["Type"])



In [ ]:
snipetsByLanguage
snipetsByLanguagePercentage = copy.deepcopy(snipetsByLanguage)
snipetsByLanguageTotal = copy.deepcopy(snipetsByLanguage)

In [ ]:

for chave, lista in snipetsByLanguage.items():
    # Calcular a contagem total de elementos na lista atual
    total_elementos_lista = len(lista)

    # Calcular a contagem de elementos na lista atual
    contagem_elementos = Counter(lista)

    # Calcular a porcentagem de ocorrência de cada elemento na lista atual
    snipetsByLanguagePercentage[chave] = {elemento: f'{(ocorrencias / total_elementos_lista) * 100:.3f}' for elemento, ocorrencias in contagem_elementos.items()}
    snipetsByLanguage[chave] = dict(Counter(snipetsByLanguage[chave]))
    snipetsByLanguageTotal[chave] = (sum(Counter(snipetsByLanguage[chave]).values()))

In [ ]:
df7 = pd.DataFrame(list(snipetsByLanguage.items()), columns=['Language', 'SnippetLang'])

df8 = pd.DataFrame(list(snipetsByLanguageTotal.items()), columns=['Language', 'SnippetLangCount'])

df9 = pd.DataFrame(list(snipetsByLanguagePercentage.items()), columns=['Language', 'SnippetLangPercentage'])

df_concatenado = pd.merge(df0, df1, on='Language').merge(df2, on='Language').merge(df3, on='Language').merge(df4, on='Language')
df_concatenado = df_concatenado.merge(df5, on='Language').merge(df6, on='Language').merge(df7, on='Language').merge(df8, on='Language').merge(df9, on='Language')
df_concatenado.to_csv(jsons_file[j]+'_results_table.csv', index=False)

## Verify Snipets by language

In [ ]:
linhas = []
for coluna, valores in snipetsByLanguage.items():
    linha = {'linguagem': coluna}  # Adicione a coluna principal
    linha.update(valores)  # Adicione os valores da linguagem
    linhas.append(linha)

# Criação do DataFrame
snipetsByLanguageDF = pd.DataFrame(linhas)
snipetsByLanguageDF = snipetsByLanguageDF.T


# Exibindo o DataFrame
snipetsByLanguageDF = snipetsByLanguageDF.fillna(0)
snipetsByLanguageDF.to_csv(jsons_file[j]+'_snipetsByLanguageDF.csv')

## Snipets By Language in Percentage

In [ ]:
linhas = []
for coluna, valores in snipetsByLanguagePercentage.items():
    linha = {'linguagem': coluna}  # Adicione a coluna principal
    linha.update(valores)  # Adicione os valores da linguagem
    linhas.append(linha)

# Criação do DataFrame
snipetsByLanguagePercentageDF = pd.DataFrame(linhas)
snipetsByLanguagePercentageDF = snipetsByLanguagePercentageDF.T

# Exibindo o DataFrame
snipetsByLanguagePercentageDF = snipetsByLanguagePercentageDF.fillna(0)
snipetsByLanguagePercentageDF.to_csv(jsons_file[j]+'_snipetsByLanguagePercentageDF.csv')